In [1]:
from nltk.stem import WordNetLemmatizer
import nltk

from summarizers import get_book_descriptions
from summarizers import frequent_ngrams
from summarizers import frequent_arbitrarygrams
from summarizers import sentimental_words
from summarizers import squish
from batch_processor import get_reviews_text
from batch_processor import token_desc_to_str
from main import print_descriptions

from nltk.sentiment import SentimentIntensityAnalyzer

2022-05-07 01:01:27.271226: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-07 01:01:27.271260: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
FILTER_SPOILERS = True
ASPECTS = ["book", "story", "writing", "plot", "character", "protagonist", "relationship", "dialogue", "action", "pacing"]
SENT_THRESHOLD = 0.6
ENT_COUNT = 20

In [3]:
book_id = "3"
# harry potter and the sorcerer's stone = 3
# twilight = 41865
# going postal = 64222

In [4]:
text, title, series = get_reviews_text(book_id, FILTER_SPOILERS, "fantasy_paranormal")

In [5]:
tokenized = nltk.word_tokenize(text)

In [6]:
lemmatizer = WordNetLemmatizer()
lemmatized = [
    lemmatizer.lemmatize(word, pos.lower()[0] if pos.lower()[0] in "nvars" else "n").lower()
    for word, pos in nltk.pos_tag(tokenized)
]

In [7]:
frequent_ngrams(lemmatized, 2)

FreqDist({('harry', 'potter'): 3400, ('first', 'time'): 620, ('first', 'book'): 582, ('potter', 'series'): 416, ('potter', 'book'): 410, ('read', 'harry'): 277, ('year', 'old'): 242, ('year', 'ago'): 212, ('even', 'though'): 207, ('whole', 'series'): 203, ...})

In [8]:
frequent_ngrams(lemmatized, 3)

FreqDist({('harry', 'potter', 'series'): 405, ('harry', 'potter', 'book'): 379, ('read', 'harry', 'potter'): 271, ('love', 'harry', 'potter'): 119, ('first', 'harry', 'potter'): 77, ('first', 'time', 'read'): 65, ('never', 'get', 'old'): 49, ('harry', 'potter', 'fan'): 46, ('like', 'harry', 'potter'): 37, ('11', 'year', 'old'): 37, ...})

In [9]:
frequent_arbitrarygrams(lemmatized, 2, 6, 3)

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.89it/s]


[(('harry', 'potter', 'and', 'the', 'sorcerer', 'stone'), 208),
 (('read', 'the', 'rest', 'of', 'the', 'series'), 82),
 (('to', 'read', 'the', 'rest', 'of', 'the'), 74),
 (('the', 'first', 'time', 'i', 'read'), 116),
 (('ca', 'wait', 'to', 'read', 'the'), 79),
 (('first', 'time', 'i', 'read', 'it'), 74),
 (('first', 'harry', 'potter', 'book'), 55),
 (('time', 'read', 'harry', 'potter'), 19),
 (('first', 'read', 'harry', 'potter'), 18),
 (('harry', 'potter', 'series'), 413),
 (('love', 'harry', 'potter'), 124),
 (('never', 'get', 'old'), 49),
 (('year', 'old'), 242),
 (('year', 'ago'), 212),
 (('even', 'though'), 207)]

In [10]:
arbitrarygrams = frequent_arbitrarygrams(lemmatized, 2, 6, 40)

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.14s/it]


In [11]:
sia = SentimentIntensityAnalyzer()
for ambigram, freq in arbitrarygrams:
    if sia.polarity_scores(" ".join(ambigram))["compound"] > 0.6:
        print(ambigram, freq, len(ambigram))

('i', 'love', 'this', 'book', 'so', 'much') 23 6
('one', 'of', 'the', 'best', 'book', 'i') 21 6
('i', 'fell', 'in', 'love', 'with') 47 5
('fell', 'in', 'love', 'with', 'the') 33 5
('be', 'one', 'of', 'the', 'best') 32 5
('i', 'love', 'the', 'harry', 'potter') 27 5
('love', 'the', 'harry', 'potter', 'series') 25 5
('perfect', 'perfect', 'perfect', 'perfect') 10 4
('best', 'book', 'series', 'ever') 6 4
('best', 'harry', 'potter', 'book') 5 4
('love', 'rowling', 'write', 'style') 4 4
('next', 'great', 'adventure') 22 3
('love', 'every', 'minute') 12 3
('perfectly', 'normal', 'thank') 11 3
('great', 'book') 126 2
('still', 'love') 91 2
('absolutely', 'love') 78 2
('great', 'read') 59 2


In [12]:
sentimental_words(lemmatized, 20, 0.5)

[('love', 3147, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6369}),
 ('great', 1121, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6249}),
 ('amazing', 637, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5859}),
 ('fun', 599, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5106}),
 ('best', 543, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6369}),
 ('wonderful', 389, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719}),
 ('awesome', 261, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6249}),
 ('fantastic', 250, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5574}),
 ('loved', 236, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5994}),
 ('perfect', 232, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5719}),
 ('amaze', 230, {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5423}),
 ('bad', 211, {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.5423}),
 ('evil', 206, {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.6597}),
 ('kind', 203,

In [13]:
print("chars", len(text))
print("words", len(text.split()))

chars 2763344
words 510745


In [ ]:
result = get_book_descriptions(book_id, text, ENT_COUNT, ASPECTS, SENT_THRESHOLD, title, series)
names_mapping, character_descriptions, aspect_descriptions, sentimental_descriptions, doc = result

In [ ]:
names_mapping

In [ ]:
extra_stopwords = {
    "character", "characters", "writer", "author", "one", "first", "second", "last"
    "whole", "book", "books", "series", "man", "woman", "place", "next",
    "able", "read", "reader", "main", "many", "previous", "1st", "2nd", "3rd", "4th", "6th", "7th",
    "protagonist", "girl", "boy", "male", "female", "great", "hero", "heroine",
    "year", "years", "old", "kid", "son"
}

print_descriptions(character_descriptions, min_descs=10, extra_stopwords=extra_stopwords)
print_descriptions(aspect_descriptions, min_descs=10, extra_stopwords=extra_stopwords)

In [ ]:
forbidden_sentimentals = {
    "thing", "things", "one", "way", "part", "deal", "job", "first",
    "book", "books", "serie", "series", "story", "stories", "read",
    "literature", "sequel"
}
print_descriptions(sentimental_descriptions, min_descs=10, extra_stopwords=forbidden_sentimentals)

In [ ]:
character_desc_count = sum(len(word_descs[1]) for word_descs in character_descriptions.items())
aspect_desc_count = sum(len(word_descs[1]) for word_descs in aspect_descriptions.items())
sentimental_desc_count = sum(len(word_descs[1]) for word_descs in sentimental_descriptions.items())
total = len(list(doc.sents))

print("characters descriptions", character_desc_count)
print("aspects descriptions", aspect_desc_count)
print("sentimental descriptions", sentimental_desc_count)
print("total review sentences", total)
print()

sum_ = character_desc_count + aspect_desc_count + sentimental_desc_count

print("% of sentences yielding a description", f"{sum_ / total * 100:.2f} %")
print()

count = 0
unique_descs = 0
word_descs = 0
for descss in [character_descriptions, aspect_descriptions, sentimental_descriptions]:
    for key, descs in descss.items():
        count += len(descs)
        unique_descs += len(set(token_desc_to_str(desc) for desc in descs))
        word_descs += sum(len(desc) for desc in descs)

print("total descriptions", count)
print("total unique descriptions", unique_descs)
print("total words in descriptsion", word_descs)

print("total words in reviews", len(doc))

print()
print("% of words in descriptions", f"{word_descs / len(doc) * 100:.2f} %")